In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import yaml
import xarray as xr

#My functions
sys.path.insert(0,'../functions/')
import functions_HeatWavesCities as fun_HWC


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:   path_main = file.read()
with open('../path_EOBS.txt', 'r') as file:   path_eobs = file.read()
    
dir_EOBS    = dir_EOBS
dir_JJA     = f'{path_main}Data/EOBS/JJA/'
dir_Yearmax = f'{path_main}Data/EOBS/Yearmax/'
if not os.path.exists(dir_JJA):      os.mkdir(dir_JJA)
if not os.path.exists(dir_Yearmax):  os.mkdir(dir_Yearmax)
    

## Calculation

In [ ]:
#Define scenarios and variables
variables = ['tx', 'tn']

time_sel = slice('1981', '2010')

#Loop over variables
for variab in variables:

    print(" -" + variab, end='')

    #Get file names
    fname = dir_EOBS + variab + '_ens_mean_0.1deg_reg_v22.0e.nc'
    data = xr.open_dataset(fname)

    #Select time
    data = data.sel(time=time_sel)
    
    #Read calendar
    with xr.open_dataset(fname, decode_times=False) as ds:
        calendar = ds.time.attrs['calendar']
        ds.close()            

    #Drop unnecessary variables
    vars_drop = set(data.data_vars).difference([variab])
    for var in vars_drop:  data = data.drop(var) 

    #Select summer
    sel_JJA  = (data.time.dt.month>=6) & (data.time.dt.month<=8)
    data_JJA = data.isel(time=sel_JJA)

    #Get climatology for summer months
    month_sta = 6
    month_end = 8
    win_half  = 15
    climatology = fun_HWC.get_clim_or_quant('climatology', data, calendar, [variab], month_sta, month_end, win_half)

    #Select original data in JJA and adjust time (for matching DOY)
    data_JJA = fun_HWC.replace_time(data_JJA, calendar, month_sta, month_end)
    
    #Calculate JJA anomalies
    JJA_anom = data_JJA.groupby("time.dayofyear") - climatology.groupby("time.dayofyear").mean()

    #Calculate summer mean, median, Q90, Q99
    data_JJA_mean = data_JJA.mean('time')
    data_JJA_Q50  = data_JJA.quantile(0.50, 'time')
    data_JJA_Q90  = data_JJA.quantile(0.90, 'time')
    data_JJA_Q99  = data_JJA.quantile(0.99, 'time')

    #Calculate summer variability
    JJA_anom_Q25 = JJA_anom.quantile(0.25, 'time')
    JJA_anom_Q75 = JJA_anom.quantile(0.75, 'time')
    JJA_anom_IQR = JJA_anom_Q75 - JJA_anom_Q25

    #Calculate summer average, yearly maximum, and summer variability
    data_Yearmax = data.resample(time='1Y').max()            

    #Creat output directory
    dir_JJA_out  =  dir_JJA + variab + '/'
    dir_Ymax_out =  dir_Yearmax + variab + '/'
    if not os.path.exists(dir_JJA_out):   os.mkdir(dir_JJA_out)
    if not os.path.exists(dir_Ymax_out):  os.mkdir(dir_Ymax_out)

    #Create output file names
    output_str =  "_EOBS_day_" + time_sel.start + "-" + time_sel.stop + ".nc"
    fname_JJA_mean = dir_JJA_out + variab + "_JJA-mean" + output_str
    fname_JJA_Q50  = dir_JJA_out + variab + "_JJA-Q50" + output_str
    fname_JJA_Q90  = dir_JJA_out + variab + "_JJA-Q90" + output_str
    fname_JJA_Q99  = dir_JJA_out + variab + "_JJA-Q99" + output_str
    fname_JJQ_IQR  = dir_JJA_out + variab + "-anom_JJA-IQR" + output_str
    fname_Yearmax  = dir_Ymax_out + variab + "_Yearmax" +  output_str

    #Save in file
    data_JJA_mean.to_netcdf(fname_JJA_mean)
    data_JJA_Q50.to_netcdf(fname_JJA_Q50)
    data_JJA_Q90.to_netcdf(fname_JJA_Q90)
    data_JJA_Q99.to_netcdf(fname_JJA_Q99)
    JJA_anom_IQR.to_netcdf(fname_JJQ_IQR)
    data_Yearmax.to_netcdf(fname_Yearmax)
